# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 13 2022 10:20AM,248538,10,PRF-39241,Bio-PRF,Released
1,Oct 13 2022 10:20AM,248538,10,PRF-39244,Bio-PRF,Released
2,Oct 13 2022 10:20AM,248538,10,PRF-39247,Bio-PRF,Released
3,Oct 13 2022 10:20AM,248538,10,PRF-39251,Bio-PRF,Released
4,Oct 13 2022 10:20AM,248538,10,PRF-39255,Bio-PRF,Released
5,Oct 13 2022 10:20AM,248538,10,PRF-39259,Bio-PRF,Released
6,Oct 13 2022 10:20AM,248538,10,PRF-39262,Bio-PRF,Released
7,Oct 13 2022 10:20AM,248538,10,PRF-39268,Bio-PRF,Released
8,Oct 13 2022 10:20AM,248538,10,PRF-39271,Bio-PRF,Released
9,Oct 13 2022 10:20AM,248538,10,PRF-39279,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,248534,Released,19
22,248535,Released,1
23,248536,Released,1
24,248537,Released,2
25,248538,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248534,NaN,19.0
248535,NaN,1.0
248536,NaN,1.0
248537,NaN,2.0
248538,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248083,1.0,0.0
248087,0.0,1.0
248089,0.0,1.0
248091,0.0,1.0
248447,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248083,1,0
248087,0,1
248089,0,1
248091,0,1
248447,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248083,1,0
1,248087,0,1
2,248089,0,1
3,248091,0,1
4,248447,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248083,1,
1,248087,,1
2,248089,,1
3,248091,,1
4,248447,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 13 2022 10:20AM,248538,10,Bio-PRF
17,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.
19,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.
38,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc."
39,Oct 13 2022 10:04AM,248535,10,Methapharm-G
40,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc."
41,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC"
42,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation
68,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation
117,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 13 2022 10:20AM,248538,10,Bio-PRF,,17
1,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,,2
2,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,,19
3,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",,1
4,Oct 13 2022 10:04AM,248535,10,Methapharm-G,,1
5,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",,1
6,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",,1
7,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,,26
8,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,28,21
9,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:20AM,248538,10,Bio-PRF,17,
1,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,2,
2,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,19,
3,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1,
4,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1,
5,Oct 13 2022 9:55AM,248533,10,"Methapharm, Inc.",1,
6,Oct 13 2022 9:47AM,248532,19,"NAPP Technologies, LLC",1,
7,Oct 13 2022 9:46AM,248531,10,ISDIN Corporation,26,
8,Oct 13 2022 9:25AM,248530,10,ISDIN Corporation,21,28
9,Oct 13 2022 9:11AM,248529,10,ISDIN Corporation,15,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:20AM,248538,10,Bio-PRF,17,
1,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,2,
2,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,19,
3,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1,
4,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:20AM,248538,10,Bio-PRF,17.0,NaN
1,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,2.0,NaN
2,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,19.0,NaN
3,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1.0,NaN
4,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 13 2022 10:20AM,248538,10,Bio-PRF,17.0,0.0
1,Oct 13 2022 10:19AM,248537,10,Eywa Pharma Inc.,2.0,0.0
2,Oct 13 2022 10:17AM,248534,10,Eywa Pharma Inc.,19.0,0.0
3,Oct 13 2022 10:05AM,248536,10,"Methapharm, Inc.",1.0,0.0
4,Oct 13 2022 10:04AM,248535,10,Methapharm-G,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3230708,125.0,32.0
18,248462,1.0,0.0
19,994047,3.0,3.0
21,1240871,3.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3230708,125.0,32.0
1,18,248462,1.0,0.0
2,19,994047,3.0,3.0
3,21,1240871,3.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,125.0,32.0
1,18,1.0,0.0
2,19,3.0,3.0
3,21,3.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,125.0
1,18,Released,1.0
2,19,Released,3.0
3,21,Released,3.0
4,10,Executing,32.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,18,19,21
Status,,,,
Executing,32.0,0.0,3.0,2.0
Released,125.0,1.0,3.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,18,19,21
0,Executing,32.0,0.0,3.0,2.0
1,Released,125.0,1.0,3.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,18,19,21
0,Executing,32.0,0.0,3.0,2.0
1,Released,125.0,1.0,3.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()